In [5]:
import keras

In [6]:
import numpy as np
import tensorflow as tf

In [7]:
from tensorflow.keras.layers import Input,GRU,Dense,Embedding,Bidirectional

In [8]:
from keras.models import Model

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [10]:
inputs = []
outputs = []

data_file = open('spa.txt', encoding='utf-8')

count = 0
for line in data_file:
    count += 1
    if count > 10:
        break
    if '\t' not in line:
        continue
    ip, temp_op, extra = line.rstrip().split('\t')
    op = temp_op
    inputs.append(ip)
    outputs.append(op)

In [11]:
inputs=[sentence.lower() for sentence in inputs]
outputs=[sentence.lower() for sentence in outputs]

In [12]:
outputs = ["<start>"+" "+sentence+" "+"<end>" for sentence in outputs]

In [13]:
EnglishTokenizer=Tokenizer(oov_token="<UNK>")
EnglishTokenizer.fit_on_texts(inputs)
inp_sequences=EnglishTokenizer.texts_to_sequences(inputs)
max_inp_len=max(len(i) for i in inp_sequences)
src_sequences=pad_sequences(inp_sequences,maxlen=max_inp_len,padding="post")
Englishword2index=EnglishTokenizer.word_index
Englishindex2word=EnglishTokenizer.index_word

In [14]:
SpanishTokenizer=Tokenizer(oov_token="<UNK>")
SpanishTokenizer.fit_on_texts(outputs)
op_sequences=SpanishTokenizer.texts_to_sequences(outputs)
max_tar_len=max(len(i) for i in op_sequences)
tar_sequences=pad_sequences(op_sequences,maxlen=max_tar_len,padding="post")
Spanishword2index=SpanishTokenizer.word_index
Spanishindex2word=SpanishTokenizer.index_word

In [15]:
src_vocab_size=len(Englishword2index)+1
trg_vocab_size=len(Spanishword2index)+1
print("src_vocab_size:",src_vocab_size)
print("tar_vocab_size:",trg_vocab_size)

src_vocab_size: 5
tar_vocab_size: 14


In [16]:
print("max_inp_len:",max_inp_len)
print("max_tar_len:",max_tar_len)

max_inp_len: 1
max_tar_len: 3


In [17]:
gru_units=100
embed_size=100

In [34]:
from numpy import asarray, zeros

embeddings_dict = dict()

glove_file = open('glove.6B.100d.txt', encoding='utf8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector = asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector

glove_file.close()

In [35]:
embedding_matrix = zeros((src_vocab_size,embed_size))

for word, index in Englishword2index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [18]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        query,values=inputs
        query_with_time_axis = tf.expand_dims(query, 1)
        score1=self.W1(query_with_time_axis)
        score2=self.W2(values)
        combined_score=tf.nn.tanh(score1 + score2)
        score = self.V(combined_score)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

In [19]:
encoder_inputs=Input(shape=(max_inp_len,))
encoder_embed=Embedding(src_vocab_size,embed_size)(encoder_inputs)
encoder_gru=Bidirectional(GRU(gru_units,return_sequences=True,return_state=True))
encoder_op,forward,backward=encoder_gru(encoder_embed)
encoder_dense=Dense(gru_units)
hidden=tf.nn.tanh(encoder_dense((tf.concat([forward,backward], axis = -1))))

decoder_inputs=Input(shape=(max_tar_len,))
decoder_embed=Embedding(trg_vocab_size,embed_size)(decoder_inputs)
attention=BahdanauAttention(gru_units)
context_vector=attention([hidden,encoder_op])
context_vector=tf.expand_dims(context_vector,1)
context_vector=tf.tile(context_vector,[1,tf.shape(decoder_embed)[1],1])
encoder_op=tf.transpose(encoder_op,perm=(0,2,1))
decoder_combined=tf.matmul(context_vector,encoder_op)
weighted=tf.concat([decoder_embed, context_vector],axis=2)

decoder_gru=GRU(gru_units,return_sequences=True,return_state=True)
decoder_op,_=decoder_gru(weighted,initial_state=hidden)

ouput=tf.concat([decoder_op,decoder_combined,decoder_embed],axis=2)

dec_op=Dense(trg_vocab_size)(ouput)

model=Model([encoder_inputs,decoder_inputs],dec_op)

In [37]:
from keras.layers import Dropout

In [38]:
encoder_inputs=Input(shape=(max_inp_len,))
encoder_embed=Embedding(src_vocab_size,embed_size,weights=[embedding_matrix])(encoder_inputs)
encoder_gru=Bidirectional(GRU(gru_units,return_sequences=True,return_state=True))
encoder_op,forward,backward=encoder_gru(encoder_embed)
encoder_dropout=Dropout(rate=0.4)
encoder_op=encoder_dropout(encoder_op)
encoder_dense=Dense(gru_units)
hidden=encoder_dense((tf.concat([forward,backward], axis = -1)))

decoder_inputs=Input(shape=(max_tar_len,))
decoder_embed=Embedding(trg_vocab_size,embed_size)(decoder_inputs)
attention=BahdanauAttention(gru_units)
context_vector=attention([hidden,encoder_op])
context_vector=tf.expand_dims(context_vector,1)
context_vector=tf.tile(context_vector,[1,tf.shape(decoder_embed)[1],1])
weighted=tf.concat([decoder_embed, context_vector],axis=2)
decoder_gru=GRU(gru_units,return_sequences=True,return_state=True)
decoder_op,_=decoder_gru(weighted,initial_state=hidden)
decoder_dropout=Dropout(rate=0.4)
dec_op=decoder_dropout(decoder_op)
ouput=tf.concat([decoder_op,weighted],axis=2)
dec_op=Dense(trg_vocab_size)(ouput)

model=Model([encoder_inputs,decoder_inputs],dec_op)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 100)               500       ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  [(None, 1, 200),             121200    ['embedding[0][0]']           
 al)                          (None, 100),                                                        
                              (None, 100)]                                                        
                                                                                              

In [21]:
trg_one_hot=tf.one_hot(tar_sequences,depth=trg_vocab_size)

In [22]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
model.fit([src_sequences,tar_sequences],trg_one_hot,epochs=10)

Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 9.2703 - accuracy: 0.0333
Epoch 2/10
1/1 [==============================] - 0s 32ms/step - loss: 8.7427 - accuracy: 0.0667
Epoch 3/10
1/1 [==============================] - 0s 27ms/step - loss: 8.4615 - accuracy: 0.0667
Epoch 4/10
1/1 [==============================] - 0s 26ms/step - loss: 9.3395 - accuracy: 0.1000
Epoch 5/10
1/1 [==============================] - 0s 27ms/step - loss: 8.6461 - accuracy: 0.1000
Epoch 6/10
1/1 [==============================] - 0s 28ms/step - loss: 8.0951 - accuracy: 0.1000
Epoch 7/10
1/1 [==============================] - 0s 28ms/step - loss: 9.6941 - accuracy: 0.1000
Epoch 8/10
1/1 [==============================] - 0s 29ms/step - loss: 12.3633 - accuracy: 0.1000
Epoch 9/10
1/1 [==============================] - 0s 29ms/step - loss: 12.8945 - accuracy: 0.1000
Epoch 10/10
1/1 [==============================] - 0s 27ms/step - loss: 12.8945 - accuracy: 0.0667
